In [1]:
from classy import Class
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
models = {}

In [3]:
base_parameters = pd.read_csv("base_parameters.csv")
base_parameters.index = ["omega_b", "omega_cdm", "theta_s_100", "tau_reio", "ln_A_s_1e10", "n_s"]
base_parameters

,TT+lowE,TE+lowE,EE+lowE,TT-TE-EE+lowE,TT-TE-EE+lowE+lensing,TT-TE-EE+lowE+lensing+BAO
omega_b,0.02212,0.02249,0.02400,0.02236,0.02237,0.02242
omega_cdm,0.12060,0.11770,0.11580,0.12020,0.12000,0.11933
theta_s_100,1.04077,1.04139,1.03999,1.04090,1.04092,1.04101
tau_reio,0.05220,0.04960,0.05270,0.05440,0.05440,0.05610
ln_A_s_1e10,3.04000,3.01800,3.05200,3.04500,3.04400,3.04700
n_s,0.96260,0.96700,0.98000,0.96490,0.96490,0.96650


In [4]:
for col in base_parameters.columns:
    params = dict(base_parameters[col])
    params["output"] = "tCl,mTk,vTk, mPk"
    params["non linear"] = "halofit",
    params["z_max_pk"] = 0.0
    models[col] = Class()
    models[col].set(params)
    models[col].compute()
    print(col + " initialized")

TT+lowE initialized
TE+lowE initialized
EE+lowE initialized
TT-TE-EE+lowE initialized
TT-TE-EE+lowE+lensing initialized
TT-TE-EE+lowE+lensing+BAO initialized


In [5]:
derived = pd.DataFrame(columns=base_parameters.columns)

In [6]:
for name, model in models.items():
    
    h = model.h()
    sigma8 = model.get_current_derived_parameters(["sigma8"])["sigma8"]
    omega_m = model.get_current_derived_parameters(["omega_m"])["omega_m"]
    
    # k_D calculations
    omega_b = model.omega_b()
    omega_m_ = omega_m / h**2
    Xe = model.get_thermodynamics()['x_e'][-1]
    z_rec = model.get_current_derived_parameters(['z_rec'])['z_rec']
    tau = model.get_current_derived_parameters(['tau_reio'])["tau_reio"]
    
    #? 100theta_s,eq Calculations
    background = model.get_background()
    z_array = background['z']
    z_eq = model.get_current_derived_parameters(['z_eq'])['z_eq']
    idx = np.abs(z_array - z_eq).argmin()
    rs_eq = background['comov.snd.hrz.'][idx]  # In Mpc
    da_eq = background['ang.diam.dist.'][idx]
    
    derived_params = {
        "H0": model.get_current_derived_parameters(["H0"])["H0"],
        "Omega_lambda": 1 - omega_m / h**2,
        "Omega_m": omega_m / h**2,
        "Omega_m (h^2)": omega_m,
        "Omega_m (h^3)": omega_m * h,
        "Sigma8": sigma8,
        "S8": sigma8 * (omega_m / 0.3) ** 0.5,
        "Sigma8 (omega_m)^(0.25)": sigma8 * (omega_m**(0.25)),
        "z_re": model.z_reio(),
        "1e9 A_s": model.get_current_derived_parameters(['A_s'])['A_s'] * 1e9,
        "1e9 A_s e^-2tau": model.get_current_derived_parameters(['A_s'])['A_s'] * 1e9 * np.exp(-2 * tau),
        "Age [Gyr]": model.age(),
        "z_star": z_rec,
        "r_star": model.get_current_derived_parameters(['rs_rec'])['rs_rec'],
        "100theta_star": model.theta_star_100(),
        "z_drag": model.get_current_derived_parameters(['z_d'])['z_d'],
        "r_drag [Mpc]": model.rs_drag(),
        "k_D [Mpc^-1]": 5.7 * (omega_m)**(-3/4) * (omega_b / omega_m_)**(-1/2) * (Xe)**(-1/2) * (1 + z_rec)**(-5/4),
        "z_eq": model.z_eq(),
        "k_eq": model.k_eq(),
        "100theta_s,eq":  100*rs_eq / (da_eq * (1+z_eq))
    }
    derived[name] = derived_params.values()
    derived.index = derived_params.keys()

In [7]:
derived

,TT+lowE,TE+lowE,EE+lowE,TT-TE-EE+lowE,TT-TE-EE+lowE+lensing,TT-TE-EE+lowE+lensing+BAO
H0,67.057179,68.635837,70.221045,67.455893,67.542540,67.856459
Omega_lambda,0.682609,0.702412,0.716487,0.686702,0.687922,0.692149
Omega_m,0.317391,0.297588,0.283513,0.313298,0.312078,0.307851
Omega_m (h^2),0.142720,0.140190,0.139800,0.142560,0.142370,0.141750
Omega_m (h^3),0.095704,0.096221,0.098169,0.096165,0.096160,0.096187
Sigma8,0.823707,0.805473,0.808800,0.824298,0.823206,0.822584
S8,0.568139,0.550616,0.552121,0.568228,0.567097,0.565433
Sigma8 (omega_m)^(0.25),0.506284,0.492867,0.494559,0.506505,0.505665,0.504732
z_re,7.517746,7.126083,7.109238,7.684799,7.679184,7.829391
1e9 A_s,2.090524,2.045035,2.115762,2.101003,2.098903,2.105209


In [8]:
table = pd.concat([base_parameters, derived])
table

,TT+lowE,TE+lowE,EE+lowE,TT-TE-EE+lowE,TT-TE-EE+lowE+lensing,TT-TE-EE+lowE+lensing+BAO
omega_b,0.022120,0.022490,0.024000,0.022360,0.022370,0.022420
omega_cdm,0.120600,0.117700,0.115800,0.120200,0.120000,0.119330
theta_s_100,1.040770,1.041390,1.039990,1.040900,1.040920,1.041010
tau_reio,0.052200,0.049600,0.052700,0.054400,0.054400,0.056100
ln_A_s_1e10,3.040000,3.018000,3.052000,3.045000,3.044000,3.047000
n_s,0.962600,0.967000,0.980000,0.964900,0.964900,0.966500
H0,67.057179,68.635837,70.221045,67.455893,67.542540,67.856459
Omega_lambda,0.682609,0.702412,0.716487,0.686702,0.687922,0.692149
Omega_m,0.317391,0.297588,0.283513,0.313298,0.312078,0.307851
Omega_m (h^2),0.142720,0.140190,0.139800,0.142560,0.142370,0.141750


In [9]:
%%latex
$\Omega_b, \Omega_{cdm}, 100\theta_{s}, \tau_{reio}, ln(10^{10}A_s), n_s, H_0, \Omega_{\Lambda}, \Omega_m, \Omega_m (h^2), \Omega_m (h^3)$
$\sigma_8, S_8, \sigma_8 (\Omega_m)^{0.25}, z_{re}, 10^9 A_s, 10^9 A_s e^{-2\tau}, Age [Gyr], z_{\ast}, r_{\ast}, 100\theta_{\star}$
$ z_{drag}, r_{drag} [Mpc], k_D [Mpc^{-1}], z_{eq}, k_{eq}, 100\theta_{s,eq}$

<IPython.core.display.Latex object>

In [10]:
#? Converting to Latex
s = r"$\Omega_b$, $\Omega_{cdm}$, $100\theta_{s}$, $\tau_{reio}$, $ln(10^{10}A_s)$, $n_s$, $H_0$, $\Omega_{\Lambda}$, $\Omega_m$, $\Omega_m (h^2)$, $\Omega_m (h^3)$, $\sigma_8$, $S_8$, $\sigma_8 (\Omega_m)^{0.25}$, $z_{re}$, $10^9 A_s$, $10^9 A_s e^{-2\tau}$, Age [Gyr], $z_{\ast}$, $r_{\ast}$, $100\theta_{\star}$, $z_{drag}$, $r_{drag}$ [Mpc], $k_D [Mpc^{-1}]$, $z_{eq}$, $k_{eq}$, $100\theta_{s,eq}$"
s = s.split(", ")
table.index = s
with open("table2.tex", 'w') as file:
    file.write(table.to_latex())

#### This saves the data to "table2.tex" which is then edited further to generate the table